##### Coursera Project: 
# Neighborhoods in Toronto
## Segmenting and Clustering

### 1.
### a) Scrape the data from Wikipedia
### b) Transform into a dataframe

In [10]:
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize

In [64]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)
text = r.text
soup = BeautifulSoup(text)


In [28]:
df = pd.read_html(url, header=0)[0]

In [29]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [31]:
df.drop(df.index[df['Borough']== 'Not assigned'], inplace=True)
df = df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [32]:
df.shape

(103, 3)

### 2. Get coordinates of each neighbourhood

#### Import geospatial data from csv file:

In [40]:
geo_data = 'https://cocl.us/Geospatial_data'
csv_file = pd.read_csv(geo_data)

In [41]:
csv_file.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Join Dataframes:

In [42]:
df_coord = df.merge(csv_file, on='Postal Code')
df_coord.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### 3. Explore and cluster the neighbourhoods

#### Create a map of Greater Toronto:

In [56]:
import folium

In [57]:
latitude = df_coord['Latitude'].mean()
longitude = df_coord['Longitude'].mean()
print('The geographical coordinates of Toronto are ', latitude,'and', longitude)

The geographical coordinates of Toronto are  43.70460773398059 and -79.39715291165048


In [59]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers
for lat, lng, label in zip(df_coord['Latitude'], df_coord['Longitude'], df_coord['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

#### I decide to concentrate on Downtown Toronto.
#### Therefore I extract the Neighbourhoods in Downtown Toronto:

In [60]:
toronto_dt = df_coord[df_coord['Borough']=='Downtown Toronto'].reset_index(drop=True)
toronto_dt.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


#### Find the geographical coordinate of Downtown Toronto as Mean of Latitude and Longitude:

In [61]:
latitude = toronto_dt['Latitude'].mean()
longitude = toronto_dt['Longitude'].mean()
print('The geographical coordinates of Downtown Toronto are ', latitude,'and', longitude)

The geographical coordinates of Downtown Toronto are  43.65459717894736 and -79.38397156842105


#### Create a map of Downtown Toronto:

In [62]:
map_toronto_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers
for lat, lng, label in zip(toronto_dt['Latitude'], toronto_dt['Longitude'], toronto_dt['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_downtown)

map_toronto_downtown